Arabidopsis seedlings are incubated with certain pharmaceuticals that attack the cell wall of the plants. This leads to increased production of lignin, a polymer that causes the seedlings to "stiffen" so that nothing can get through the cell wall anymore. For analysis, we use a specific dye that docks onto the individual lignin polymers, making them visible. And this is exactly where our interest lies. Using Fiji or ImageJ, we have to manually quantify the stained regions for every sample we analyse, sometimes several hundreds in number. Instead, we would like to have a tool that can be fed with all the images and quantifies the stained regions within seconds.

<img src="images/root0001.png" width="400" height="200">

First, all required packages for running the notebook are installed. Lines starting with an "#" are comments and will not be executed when running the respective cell.

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import pandas as pd
import xlsxwriter
# import napari

The following cell contains definitions of all methods, which will be used for downstream analysis. You can find descriptions of the functionalities of the methods at the top of the method definitions.

In [4]:
# Classes and functions

class RootAnalysis():
    '''
    Object for quantifying the amount of lignin polymers in Arabidopsis seedlings
    methods: read, to_gray, crop_img, calc_thresh, bin_img, smoothen, calc_intensity, save_img
    '''
    def __init__(self, file):
        '''
        Class initialization
        f: path to the file containing all images that are to be analysed, 
        e.g., 'images/root_images/root0004.tif'
        '''
        self.f = file
    
    def read(self):
        '''
        Method for reading images
        '''
        # Read image
        img = cv.imread(self.f)
        return img
    
    def to_gray(self, img):
        '''
        Method for converting images to gray-scale
        img: image to be converted to gray-scale
        '''
        # Convert image to grayscale
        img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        return img_gray
    
    def crop_img(self, img, n=255):
        '''
        Method for "cropping" images by setting the first and last 255 pixel rows to white
        img: image to be cropped
        n: number of top and bottom pixel rows to be cropped
        '''
        # Create copy of image
        img_cp = img.copy()
        # Set first and last 255 rows to white
        img_cp[:225] = n
        img_cp[-225:] = n
        return img_cp
    
    def calc_thresh(self, img, n=255):
        '''
        Method for calculating the threshold that will be used to binarize images, 
        i.e., separate foreground (in this case stained regions) from background
        img: image, for which the threshold is to be calculated
        n: number of top and bottom pixel rows that are ignored
        '''
        # Calculate 10%-quantile
        q_10 = np.quantile(img[n+1:-(n+1),:],0.10)
        # Calculate threshold as mean pixel intensity over 10%-quantile
        # Adjust by some constant, e.g., c=2
        th = np.average(img[img <= q_10]) - 2
        return th
    
    def bin_img(self, img, th):
        '''
        Method for binarizing images
        img: image to be binarized
        th: threshold
        '''
        # Binarize image by means of thresholding
        _, bin_img = cv.threshold(img,th,255,cv.THRESH_BINARY)
        return bin_img
    
    def smoothen(self, img):
        '''
        Method for smoothening boundaries in binarized images
        img: binarized image
        '''
        # Smoothen boundaries in binarised image by means of morphological opening
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2))
        sm_img = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
        return sm_img
    
    def calc_intensity(self, bin_img, img, n=225):
        '''
        Method for calculating gray-scale intensities over
            1) stained region
            2) cropped region
            3) whole image
        bin_img: binarized image
        img: original image
        n: number of top and bottom pixel rows that are ignored
        '''
        # Create copy of binarized image and convert to boolean
        bool_img = bin_img.copy()
        bool_img[bool_img == 0] = True
        bool_img[bool_img == 255] = False
        bool_img = bool_img.astype('bool')

        # image dimensions, i.e., h x w pixels
        h, w = img.shape
        # area of whole image
        area = h*w
        # area of cropped region
        area_cp = (h-2*n)*w
        # area of stained region
        area_st = np.sum(bool_img)

        # grayscale intensity normalized over stained region
        int1 = np.sum(img[bool_img])/area_st
        # grayscale intensity normalized over cropped region
        int2 = np.sum(img[bool_img])/area_cp
        # grayscale intensity normalized over whole image
        int3 = np.sum(img[bool_img])/area

        # print(f"Grayscale intensity normalized over stained region: {int1}")
        # print(f"Grayscale intensity normalized over cropped region: {int2}")
        # print(f"Grayscale intensity normalized over whole image: {int3}")
        return np.array([area_st, area_cp]), np.array([int1, int2, int3])
    
    def save_img(self, img, name):
        '''
        Method for smoothening boundaries in binarized images
        img: image to be saved
        name: name suffix added to original file name
        '''
        txt = self.f.split('.')
        cv.imwrite(f"{txt[0]}_{name}.tif", img)
    

The following cell runs the analysis for one specific image. The path to the image is specified as variable f, e.g., f = 'images/root_images/root0004.tif'. If you wish to save the processed image after a specific step, e.g., after gray-scale conversion or binarization, uncomment the corresponding line by removing the "#". After successful execution of the cell, an excel file will be stored, containing the gray-scale intensity over the stained region, the cropped region, the whole image, the area of the stained region and the area of the cropped region. Currently, the file will be saved as "output.xlsx". If you wish to change the name of the output file, just enter a different filename (but keeping the file extension .xlsx).

In [99]:
# path
f = 'images/root_images/root0004.tif'

# create instance for image analysis
object = RootAnalysis(f)

# read image
img = object.read()

# convert image to grayscale
gray_img = object.to_gray(img)

# save gray-scale image
# object.save_img(gray_img, 'gray')

# crop image
cropped_img = object.crop_img(gray_img)

# save cropped image
# object.save_img(cropped_img, 'cropped')

# calculate threshold
th = object.calc_thresh(gray_img)
# Define constant threshold instead
# th = 140

# binarize image
bin_img = object.bin_img(cropped_img, th)

# smoothen boundaries in binarized image
final_img = object.smoothen(bin_img)

# # save binarized image
# object.save_img(final_img, 'binarized')

# compute area of cropped and stained region
# compute grayscale intensities normalized over stained region, cropped region, whole image
area, intensity = object.calc_intensity(final_img, gray_img)

# create dataframe
data = {'grayscale intensity stained region':[], 'grayscale intensity cropped region':[], 'grayscale whole image':[], 'area stained region':[], 'area cropped region':[]}
df = pd.DataFrame(data)

# insert values
new_row = np.concatenate((intensity, area))
df.loc[len(df)] = new_row

# export to excel
df.to_excel("output.xlsx") 

# # visualize original and processed images with napari
# viewer = napari.Viewer()
# viewer.add_image(img, rgb=True, name='Original image')
# viewer.add_image(gray_img, name='Grayscale image')
# viewer.add_image(cropped_img, name='Cropped image')
# viewer.add_image(bin_img, name='Binarized image')
# viewer.add_image(final_img, name='Binarized image w/ smoothened boundaries')

Grayscale intensity normalized over stained region: 114.23573452043706
Grayscale intensity normalized over cropped region: 3.4674294909591197
Grayscale intensity normalized over whole image: 1.4357325236002605
   grayscale intensity stained region  grayscale intensity cropped region  \
0                          114.235735                            3.467429   

   grayscale whole image  area stained region  area cropped region  
0               1.435733             325632.0               9884.0  


The following cell iterates over all images contained in the specified directory.

In [5]:
# create dataframe
data = {'grayscale intensity stained region':[], 'grayscale intensity cropped region':[], 'grayscale whole image':[], 'area stained region':[], 'area cropped region':[]}
df = pd.DataFrame(data)

# assign directory
directory = 'images/root_images'
 
# iterate over files in directory
for filename in os.listdir(directory):
    # path
    f = os.path.join(directory, filename)

    # create instance for image analysis
    object = RootAnalysis(f)

    # read image
    img = object.read()

    # convert image to grayscale
    gray_img = object.to_gray(img)
    # save gray-scale image
    # object.save_img(gray_img, 'gray')

    # crop image
    cropped_img = object.crop_img(gray_img)

    # save cropped image
    # object.save_img(cropped_img, 'cropped')

    # calculate threshold
    th = object.calc_thresh(gray_img)
    # Define constant global threshold instead
    # th = 140

    # binarize image
    bin_img = object.bin_img(cropped_img, th)

    # smoothen boundaries in binarized image
    final_img = object.smoothen(bin_img)

    # save binarized image
    # object.save_img(final_img, 'binarized')
    
    # compute area of cropped and stained region
    # compute grayscale intensities normalized over stained region, cropped region, whole image
    area, intensity = object.calc_intensity(final_img, gray_img)

    # insert values
    new_row = np.concatenate((intensity, area))
    df.loc[len(df)] = new_row

    # # visualize original and processed images with napari
    # viewer = napari.Viewer()
    # viewer.add_image(img, rgb=True, name='Original image')
    # viewer.add_image(gray_img, name='Grayscale image')
    # viewer.add_image(cropped_img, name='Cropped image')
    # viewer.add_image(bin_img, name='Binarized image')
    # viewer.add_image(final_img, name='Binarized image w/ smoothened boundaries')

# export to excel
df.to_excel("output.xlsx")